In [68]:
import pandas as pd
import numpy as np
import numpy.random as nr
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest

from scipy import stats
import scipy.spatial as ss
from scipy.special import digamma,gamma
import numpy.random as nr
import random
import matplotlib.pyplot as plt
import re
from scipy.stats.stats import pearsonr
import numpy.linalg as la
from numpy.linalg import eig, inv, norm, det
from scipy import stats
from math import log,pi,hypot,fabs,sqrt

In [45]:
start_day = datetime.datetime(2002,6,13)
end_day = datetime.datetime(2022,6,13)
dj1 = web.get_data_stooq('^DJI',start_day,end_day)
dj1.sort_index(ascending=True,inplace=True)
dj1['Close_1'] = dj1['Close'].shift(-1)

d = range(0,31)
high_lag = pd.DataFrame(dj1['High'])
for i in d[1:]:
    column = 'high_lag_'+str(i)
    high_lag[column] = dj1['High'].shift(i)
    
open_lag = pd.DataFrame(dj1['Open'])
for i in d[1:]:
    column = 'open_lag_'+str(i)
    open_lag[column] = dj1['Open'].shift(i)
    
low_lag = pd.DataFrame(dj1['Low'])
for i in d[1:]:
    column = 'low_lag_'+str(i)
    low_lag[column] = dj1['Low'].shift(i)
    
vol_lag = pd.DataFrame(dj1['Volume'])
for i in range(1,31):
    column = 'vol_lag_'+str(i)
    vol_lag[column] = dj1['Volume'].shift(i)
F = pd.concat([high_lag,open_lag,low_lag,vol_lag,dj1['Close_1']],axis=1)

In [46]:
F['Close_1']

Date
2002-06-13     9474.21
2002-06-14     9687.42
2002-06-17     9706.12
2002-06-18     9561.57
2002-06-19     9431.77
                ...   
2022-06-07    32910.90
2022-06-08    32272.79
2022-06-09    31392.79
2022-06-10    30516.74
2022-06-13         NaN
Name: Close_1, Length: 5036, dtype: float64

### Mutual Information Function

In [48]:
# calculate <nx> of each feature
def avgdigamma(points,dvec):
#This part finds number of neighbors in some radius in the marginal space
#returns expectation value of <psi(nx)>
    N = len(points)
    tree = ss.cKDTree(points)
    avg = 0.
    for i in range(N):
        dist = dvec[i]
        #subtlety, we don't include the boundary point, 
        #but we are implicitly adding 1 to kraskov def bc center point is included
        num_points = len(tree.query_ball_point(points[i],dist-1e-15,p=float('inf'))) 
        avg += digamma(num_points)/N
    return avg

def mi_Kraskov(features,target, k=3,intens = 1e-10):
    #Merge features and the target into a single dataframe
#     X_x = features
#     Y_y = target
    X = np.vstack((features.T,target.T))
    
    # adding small noise to X, e.g., x<-X+noise
    x = []
    for i in range(len(X)):
        tem = []
        for j in range(len(X[i])):
            #every element becomes list
            #tem = [ [[ X[i][j]+ + intens*nr.rand(1)[0] ],[]...[]] , [[]...[]] , ... , [[]...[]] ]
            tem.append([X[i][j] + intens*nr.rand(1)[0]])
        x.append(tem)
    #Generate tree which amount is equal to the number of features
    points = []
    for j in range(len(x[0])):
        tem = []
        for i in range(len(x)):
            tem.append(x[i][j][0])
        points.append(tem)
    tree = ss.cKDTree(np.array(points))
    
    # every element stores the distance between points
    dvec = []
    for i in range(len(x)):
        dvec.append([])
        
    for point in points:
        # Find k-nearest neighbors in joint space, p=inf means max norm
        knn = tree.query(point, k+1, p=float('inf'))
        points_knn = []
        for i in range(len(x)):
            dvec[i].append(float('-inf'))
            points_knn.append([])
        for j in range(k+1):
            for i in range(len(x)):
                points_knn[i].append(points[knn[1][j]][i])

        # Find distances to k-nearest neighbors in each marginal space
        for i in range(k+1):
                for j in range(len(x)):
                    if dvec[j][-1] < fabs(points_knn[j][i]-points_knn[j][0]):
                        dvec[j][-1] = fabs(points_knn[j][i]-points_knn[j][0])

    ret = 0.
    for i in range(len(x)):
        ret -= avgdigamma(x[i], dvec[i])
    ret += digamma(k) - (float(len(x))-1.)/float(k) + (float(len(x))-1.) * digamma(len(x[0]))
    return ret

In [49]:
def feature_selection(X, y, metric, tol=0):
    """
    A function to perform a forward feature selection based on a features to target dependence metric.
    Inputs:
        x: np.ndarray, 2D array of features

        y: np.ndarray,
       
        metric: np.ufunc,
    Output:
        selected: list, the list of selected feature indexes
    """
    selected = [] # indices of selected fetures
    rest = list(range(X.shape[1]))
    prev_score = 0
    # stop = False
    counter = 0
    while len(rest)>0 and counter<5:
        id_max = rest[0]
        max_score = 0
        counter += 1
        for id in rest:
#             nan_arrays = np.array(np.where(np.isnan(X.iloc[:,selected+[id]])))[0]
# #             print(nan_arrays)
#             if nan_arrays.size == 0:  
# #                 start_idx = np.max(nan_arrays[0]) + 1
#                 x = np.array(X.iloc[:,selected+[id]]).reshape((-1, 1))
#                 Y = np.array(y).reshape((-1,1))
#                 temp_score = metric(x, Y)
                
#             else:
#                 start_idx = np.max(nan_arrays[0]) + 1
#                 x = np.array(X.iloc[start_idx:,selected+[id]])
#                 Y = np.array(y[start_idx:]).reshape((-1,1))
            temp_score = metric(X, y).mic()
                
            if temp_score>max_score:
                id_max = id
                max_score = temp_score
        if prev_score>max_score - tol:
            return selected
        prev_score = max_score
        selected = selected+[id_max]
        rest.remove(id_max)
        print(f"rest {rest}, selected: {selected}, score: {max_score}")
    return selected

In [50]:
features = F.iloc[30:-1,:-1].copy()
target = F.iloc[30:-1,-1].copy()

In [51]:
X = np.array(features)
Y = np.array(target).reshape((-1,1))
SC_X = MinMaxScaler(feature_range=(0,1))
SC_Y = MinMaxScaler(feature_range=(0,1))
x_mic = SC_X.fit_transform(X)
y_mic = SC_Y.fit_transform(Y)

In [89]:
F.shape

(5036, 125)

In [91]:
F_scaled = F.iloc[30:-1,:].copy()
for i in range(F_scaled.shape[1]):
    if i<123:
        F_scaled.iloc[:,i] = x_mic[:,i]
    F_scaled.iloc[:,i] = y_mic.reshape((-1,))

In [94]:
from minepy import MINE,cstats

In [95]:
mine = MINE()

In [59]:
mic_c, tic_c =  cstats(x_mic.T,y_mic.T,alpha=9, c=5, est="mic_e")

In [ ]:
from ace import model
from ace import ace

In [14]:
def feature_selection(X, y, metric, tol=0):
    """
    A function to perform a forward feature selection based on a features to target dependence metric.
    Inputs:
        x: np.ndarray, 2D array of features

        y: np.ndarray,
       
        metric: np.ufunc,
    Output:
        selected: list, the list of selected feature indexes
    """
    selected = [] # indices of selected fetures
    rest = list(range(X.shape[1]))
    prev_score = 0
    # stop = False
    counter = 0
    while len(rest)>0 and counter<5:
        id_max = rest[0]
        max_score = 0
        counter += 1
        for id in rest:
            nan_arrays = np.array(np.where(np.isnan(X.iloc[:,selected+[id]])))[0]
#             print(nan_arrays)
            if nan_arrays.size == 0:  
#                 start_idx = np.max(nan_arrays[0]) + 1
                x = np.array(X.iloc[:,selected+[id]]).reshape((-1, 1))
                Y = np.array(y).reshape((-1,1))
                temp_score = metric(x, Y)
                
            else:
                start_idx = np.max(nan_arrays[0]) + 1
                x = np.array(X.iloc[start_idx:,selected+[id]])
                Y = np.array(y[start_idx:]).reshape((-1,1))
                temp_score = metric(x, Y)
                
            if temp_score>max_score:
                id_max = id
                max_score = temp_score
        if prev_score>max_score - tol:
            return selected
        prev_score = max_score
        selected = selected+[id_max]
        rest.remove(id_max)
        print(f"rest {rest}, selected: {selected}, score: {max_score}")
    return selected